In [ ]:
BreastCancer <- read.csv("wisc_bc_data.csv", stringsAsFactors = FALSE)
summary(BreastCancer)[1:4]

str(BreastCancer)

BreastCancer[1:5, 1:5]

## Data Preparation



**To see the variables and the columns
**


In [ ]:
#To see the variables and the columns
dim(BreastCancer)

In [ ]:
#Check for missing data
sum(is.na(data))

In [ ]:
table(BreastCancer$diagnosis)

Test the Benign and Malignant



In [ ]:
library(reshape2)
bc <- BreastCancer[, -1]
bc.melt <- melt(bc, id.var = "diagnosis")
head(bc.melt)
library(reshape2)
bc <- BreastCancer[, -1]
bc.melt <- melt(bc, id.var = "diagnosis")
head(bc.melt)

ggplot(data = bc.melt, aes(x = diagnosis, y = log(value + 1), fill = diagnosis)) +
    geom_boxplot() + theme_bw() + facet_wrap(~variable, ncol = 8)



In [ ]:
library(tidyverse)
data <- select(BreastCancer, -1) %>Test the Benign and Malignant

%
    mutate_at("diagnosis", as.factor)
data[1:5, 1:5]

corrplot::corrplot(cor(data[, -1]))

### Data for training and testing



Train data and test data (split data 7:3)



In [ ]:
library(sampling)
set.seed(123)
train_id <- strata(data, "diagnosis", size = rev(round(table(data$diagnosis) * 0.7)))$ID_unit
train_data <- data[train_id, ]
test_data <- data[-train_id, ]Test the Benign and Malignant



Check that the data is split to ensure the same proportion of positive and negative samples in the training and test data


In [ ]:
# View the proportion of positive and negative samples in training and test data
prop.table(table(train_data$diagnosis))

prop.table(table(test_data$diagnosis))

Train the Classification Trees model

In [ ]:
set.seed(123)
tree.pros <- rpart(diagnosis ~ ., data = train_data, method = "class")
printcp(tree.pros)
rpart.plot(tree.pros)


In [ ]:
tree.pros$cptable
plotcp(tree.pros)


In [ ]:
cp <- min(tree.pros$cptable[3, ])
prune.tree.pros <- prune(tree.pros, cp = cp)
plot(as.party(tree.pros))



In [ ]:
ctrl <- trainControl(method = "cv", number = 10)

# CV bagged model
bagged_cv <- train(diagnosis ~ ., data = train_data, method = "treebag", trControl = ctrl,
    importance = TRUE)
bagged_cv
plot(varImp(bagged_cv), 20)


In [ ]:
party.pros.test <- predict(prune.tree.pros, newdata = test_data, type = "class")
table(party.pros.test, test_data$diagnosis)
## Confusion Matrix and Statistics
confusionMatrix(party.pros.test, test_data$diagnosis, positive = "B")



